<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/filtra_prodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pacotes
import pathlib
from osgeo import ogr, gdal, gdalconst
from skimage.morphology import disk, dilation, erosion
import os
import numpy as np
import shutil
from google.colab import drive
import cv2
from datetime import date, timedelta
import pandas as pd
import pickle

In [2]:
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [ ]:
# Cortando o Prodes para a nossa região de interesse
extent=[-59.9206710380000018, -7.3125828444699836, -59.1668605229999969, -6.7566813625300162]
clip =  gdal.Warp(destNameOrDestDS = "/content/drive/My Drive/projeto_DL/prodes_tratado/corte_prodes.tif",
                  srcDSOrSrcDSTab  = "/content/drive/My Drive/projeto_DL/dados_brutos/PDigital2000_2021_AMZ_raster_v20220915.tif",
                  outputBounds     = (extent[0], extent[1], extent[2], extent[3]),
                  cropToCutline    = True,
                  copyMetadata     = True,
                  xRes = ((extent[2]-extent[0])/(256*10)),
                  yRes = ((extent[3]-extent[1])/(256*10)))
clip = None

In [3]:
# Agora criando máscara anual de floresta, não floresta e água _por ano_
# Abrindo tif cortado
dados = gdal.Open("/content/drive/My Drive/projeto_DL/prodes_tratado/corte_prodes.tif")
# Transformando em array
dados2 = dados.ReadAsArray()
dados2 = dados2.astype('int32')

In [4]:
np.unique(dados2, return_counts=True) # Veja que a região selecionada não tem nem nuvem (val = 32) nem água (val = 91)
# Por isso só vou criar 2 labels

(array([  7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,
         20,  21,  50,  51,  53,  54,  55,  56,  57,  58,  59,  60, 100,
        101], dtype=int32),
 array([ 979015,   56971,   52276,   37777,   39422,   61263,   77298,
          74049,   95040,  127366,  151376,  141334,  259325,  220445,
         249405,    4196,     211,     571,     191,    5164,     523,
            135,    1420,     494,     934, 3856515,   60884]))

In [5]:
# Criando função que converte o que é floresta e não floresta _para cada ano_
# Na nova formulação:
  # 0 = floresta
  # 1 = não-floresta
def troca_label(src_array, ano):
  # Inicializando um objeto a ser retornado depois
  out_array = np.zeros(shape = src_array.shape) # Só vamos substituir os de não-floresta
  if ano == 2016:
    lista_desmatamento_posterior = [
        # Acrescentando o que é floresta até hoje
        100,
        # 2017
        57,17,
        # 2018
        58,18,
        #2019
        59,19,
        #2020
        60,20,
        #2021
        61,21
    ]
  elif ano == 2017:
    lista_desmatamento_posterior = [
        # Acrescentando o que é floresta até hoje
        100,
        # 2018
        58,18,
        #2019
        59,19,
        #2020
        60,20,
        #2021
        61,21
    ]
  elif ano == 2018:
    lista_desmatamento_posterior = [
        # Acrescentando o que é floresta até hoje
        100,
        #2019
        59,19,
        #2020
        60,20,
        #2021
        61,21
    ]
  elif ano == 2019:
    lista_desmatamento_posterior = [
        # Acrescentando o que é floresta até hoje
        100,
        #2020
        60,20,
        #2021
        61,21
    ]
  elif ano == 2020:
    lista_desmatamento_posterior = [
        # Acrescentando o que é floresta até hoje
        100,
        #2021
        61,21
    ]
  elif ano == 2021:
    lista_desmatamento_posterior = [
        # Acrescentando o que é floresta até hoje
        100
    ]
  for i in range(src_array.shape[0]):
    for j in range(src_array.shape[1]):
      if src_array[i,j] not in lista_desmatamento_posterior:
        out_array[i,j] = 1
  return out_array

In [6]:
# Criando array de cada ano
array_2016 = troca_label(dados2, 2016)
array_2017 = troca_label(dados2, 2017)
array_2018 = troca_label(dados2, 2018)
array_2019 = troca_label(dados2, 2019)
array_2020 = troca_label(dados2, 2020)
array_2021 = troca_label(dados2, 2021)

In [7]:
print(np.unique(array_2016, return_counts=True))
print(np.unique(array_2017, return_counts=True))
print(np.unique(array_2018, return_counts=True))
print(np.unique(array_2019, return_counts=True))
print(np.unique(array_2020, return_counts=True))
print(np.unique(array_2021, return_counts=True))

(array([0., 1.]), array([4881383, 1672217]))
(array([0., 1.]), array([4729872, 1823728]))
(array([0., 1.]), array([4587118, 1966482]))
(array([0., 1.]), array([4327299, 2226301]))
(array([0., 1.]), array([4105920, 2447680]))
(array([0., 1.]), array([3856515, 2697085]))


In [8]:
# Salvando
# https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict-or-any-other-python-object
# 2016
with open("/content/drive/My Drive/projeto_DL/prodes_tratado/prodes_2016.pkl", 'wb') as handle:
    pickle.dump(array_2016, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 2017
with open("/content/drive/My Drive/projeto_DL/prodes_tratado/prodes_2017.pkl", 'wb') as handle:
    pickle.dump(array_2017, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 2018
with open("/content/drive/My Drive/projeto_DL/prodes_tratado/prodes_2018.pkl", 'wb') as handle:
    pickle.dump(array_2018, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 2019
with open("/content/drive/My Drive/projeto_DL/prodes_tratado/prodes_2019.pkl", 'wb') as handle:
    pickle.dump(array_2019, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 2020
with open("/content/drive/My Drive/projeto_DL/prodes_tratado/prodes_2020.pkl", 'wb') as handle:
    pickle.dump(array_2020, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 2021
with open("/content/drive/My Drive/projeto_DL/prodes_tratado/prodes_2021.pkl", 'wb') as handle:
    pickle.dump(array_2021, handle, protocol=pickle.HIGHEST_PROTOCOL)